In [ ]:
# This script is for trainit the model.
# For base ve use pretrained model cjvt/t5-sl-small
# https://huggingface.co/cjvt/t5-sl-small

In [ ]:
!pip install datasets
!pip install transformers 
!pip install SentencePiece 
!pip install evaluate
!pip install rouge_score
!pip install tensorflow_text
!pip install torchtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch
import numpy as np
from datasets import Dataset
import pyarrow as pa
import evaluate
from google.colab import drive
from datetime import datetime as dt
import os
from torch.utils.data import DataLoader

In [ ]:
# if the variable is set to true, then the script takes the base model (cjvt/t5-sl-small)
# If the variable is set to False, then searches in direcotry /content/gdrive/MyDrive/NLPProject/models/T5 for latest trained model
newModel = True

In [ ]:
# Reading Excel file with the sentences we are using to train the model
drive.mount('/content/gdrive/', force_remount=True)
trainDatabase = pd.read_excel("/content/gdrive/MyDrive/NLPProject/data/DeeplTrening.xlsx")

Mounted at /content/gdrive/


In [ ]:
# Seraching for path to latest build model
path = "/content/gdrive/MyDrive/NLPProject/models/T5/"

pathToModel = ""
cas = 0

# Checchk for latest build model
for filename in os.listdir(path):
    if os.path.isdir(os.path.join(path,filename)):
      if os.path.getmtime(os.path.join(path,filename)) > cas:
        cas = os.path.getmtime(os.path.join(path,filename))
        pathToModel = os.path.join(path,filename)

print(pathToModel,cas)

/content/gdrive/MyDrive/NLPProject/models/T5/2023-05-23T17:41:41.416238 1684863702.0


In [ ]:
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained("cjvt/t5-sl-small")

In [ ]:
# Preparing data

trainDatabase['input_ids'] = None
trainDatabase['attention_mask'] = None
trainDatabase['labels'] = None
trainDatabase['decoder_attention_mask'] = None

for i in trainDatabase.index:
    a = tokenizer("parafraza: "+trainDatabase.at[i,'original'], padding="max_length", truncation=True, max_length=50)
    b = tokenizer(trainDatabase.at[i,'parafraza'], padding="max_length", truncation=True, max_length=50)
    trainDatabase.at[i,'input_ids'] = a.input_ids
    trainDatabase.at[i,'attention_mask'] = a.attention_mask
    trainDatabase.at[i,'labels'] = b.input_ids
    trainDatabase.at[i,'decoder_attention_mask'] = b.attention_mask

trainDatabase = trainDatabase.drop(['original','anprevod','parafraza'],axis=1)
a = pa.Table.from_pandas(trainDatabase)
fullDataset = Dataset(a)
fullDataset.set_format("torch")

trainDataset = fullDataset.shuffle(seed=42).select(range(5000))
valDataset = fullDataset.shuffle(seed=42).select(range(1000))

train_dataloader = DataLoader(trainDataset, shuffle=True, batch_size=64)
eval_dataloader = DataLoader(valDataset, batch_size=64)

In [ ]:
# Loading model for training

from transformers import T5ForConditionalGeneration
if newModel:
  print("Training a new model.")
  model = T5ForConditionalGeneration.from_pretrained("cjvt/t5-sl-small")
else:
  print("Training existing model")
  model = T5ForConditionalGeneration.from_pretrained(pathToModel)


Training existing model


In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=3e-4, eps=1e-8)

In [ ]:
from transformers import get_scheduler

num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32000, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32000, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
# Model trainig

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    print(epoch)
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    metric = evaluate.load("bleu")
    model.eval()
    for batch in eval_dataloader:
      napovedi = []
      resnice = []
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
        outputs = model(**batch)

      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      for i in predictions:
        napovedi.append(tokenizer.decode(i.tolist()))
      predikcija = predictions[0].tolist()
      #predikcija = tf_sp.detokenize(predikcija)
      predikcija = [tokenizer.decode(predikcija)]
      zares = batch["labels"].tolist()
      for i in zares:
        resnice.append(tokenizer.decode(i))
      metric.add_batch(predictions=napovedi, references=resnice)

    rez = metric.compute()
    print(rez)

  0%|          | 0/79 [00:00<?, ?it/s]

0
{'bleu': 0.9203948470595957, 'precisions': [0.9497324176270452, 0.9245494367390057, 0.9097208819285524, 0.9003218714925747], 'brevity_penalty': 0.9994586645386073, 'length_ratio': 0.9994588110077841, 'translation_length': 110807, 'reference_length': 110867}


In [ ]:
# Now we save trained model
# The name of the directory is made in such a way that the date on which the model was made is clear

pathForModel = "/content/gdrive/MyDrive/NLPProject/models/T5/"+dt.now().isoformat()
model.save_pretrained(pathForModel, from_pt=True) 
